In [1]:
%cd "C:\Users\jigna\PredictiveAnalyticsProject"

C:\Users\jigna\PredictiveAnalyticsProject


In [2]:
import pandas as pd

# Import dataset
data = pd.read_csv('data.csv')

In [13]:
data.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
deposit      object
dtype: object

In [6]:
# Check for missing values
data.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [7]:
y = data['deposit']
X = data.drop(columns=['deposit'])

In [8]:
# Use sklearn to create train and test data sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)